<a href="https://colab.research.google.com/github/Henil21/Twitter_Climate_Change/blob/main/Twitter_Climate_Change.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Getting Data Ready 🌴

In [2]:
Data=pd.read_csv('/content/drive/MyDrive/twitter_sentiment_data.csv')
Data['sentiment'] = Data['sentiment'].replace(-1, 3)

# Write the modified DataFrame back to a new CSV file or overwrite the existing one


In [3]:

message=Data['message']
message[10]
sentiment=Data['sentiment']
print(f"message\n{message[10]}\n\nsentiment : {sentiment[10]}\n\n")
print(f"number of data : {len(message)}")

message
RT @DrDeJarnett: It's vital that the public health community addresses climate change- via @Climate4Health's Tabola #APHA2016 https://t.co/Ã¢â‚¬Â¦

sentiment : 1


number of data : 43943


In [4]:
sentiment.value_counts()

1    22962
2     9276
0     7715
3     3990
Name: sentiment, dtype: int64

## Visualizing random tweet 🎋

In [5]:
seed_number=random.randrange(0,len(message)-5)
for i in range(seed_number,seed_number+5):
    print(f"T_number : {i}\n message : {message[i]}\n Sentiment : {sentiment[i]}\n")
    print('-----------------------------------------------------------------------------------------------')


T_number : 11994
 message : RT @MichaelEMann: Oh my that's a low bar you've set Elon...
RT @elonmusk Tillerson also said that “the risk of climate change does exist”
 Sentiment : 1

-----------------------------------------------------------------------------------------------
T_number : 11995
 message : RT @davidcicilline: RT this to remind @POTUS @realDonaldTrump that 2016 was the hottest year on record and climate change is real. https://…
 Sentiment : 1

-----------------------------------------------------------------------------------------------
T_number : 11996
 message : Concerns global warming 'worse than thought' https://t.co/5xpNmACh4P https://t.co/TM5oF9ZZ7J
 Sentiment : 1

-----------------------------------------------------------------------------------------------
T_number : 11997
 message : @realDonaldTrump it's too bad you're making most Americans worried about the future of global warming.
 Sentiment : 1

-------------------------------------------------------------

## Splitting data 🌳

In [6]:
from sklearn.model_selection import train_test_split

train_sentences,val_sentences,train_labels,val_labels=train_test_split(Data["message"].to_numpy(),
                                                                       Data["sentiment"].to_numpy(),
                                                                       test_size=0.3,
                                                                       random_state=42)
print(f"train_sentences : {len(train_sentences)}\nval_sentences : {len(val_sentences)}\n")


train_sentences : 30760
val_sentences : 13183



In [7]:
# find the average number of token(words) in the training tweet
# Find average number of tokens (words) in training Tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

17

In [8]:

from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Setup text vectorization with custom variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 17# max length our sequences will be (e.g. how many words from a Tweet does our model see?)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                     split="whitespace",
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [9]:
text_vectorizer.adapt(train_sentences)
sample_text='the goverment should propose a uniform policy for climate change'
text_vectorizer([sample_text])

<tf.Tensor: shape=(1, 17), dtype=int64, numpy=
array([[   5,    1,  108, 5630,    9,    1,  252,   15,    2,    3,    0,
           0,    0,    0,    0,    0,    0]])>

In [10]:
words_in_voc=text_vectorizer.get_vocabulary()
top_5=words_in_voc[:5]
bottom_5=words_in_voc[-5:]
print(f"number of word in vocab {len(words_in_voc)}\n")
print(f"5 most comman word in  vocab {top_5}\n")
print(f"5 least comman  word in vocab {bottom_5}")


number of word in vocab 10000

5 most comman word in  vocab ['', '[UNK]', 'climate', 'change', 'rt']

5 least comman  word in vocab ['expressed', 'exponentially', 'exploitation', 'explicitly', 'expertise']


In [11]:
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,#set the input shape
                             output_dim=128,
                             input_length=max_length)

In [12]:

# Convert text into number
from sklearn.feature_extraction.text import TfidfVectorizer
# our model
from sklearn.naive_bayes import MultinomialNB

from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),# convert words to number using tfidf
    ("clf",MultinomialNB())# model the text
])
# fit the pipleine to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [13]:

baseline_score=model_0.score(val_sentences,val_labels)
# as we use .evaluate in tf for sklearn its .score
baseline_score*100

57.809299855875

In [14]:
train_sentences

array(['@ErrataRob @pvineetha seeing a direct threat to seeds but this is an indication of climate change, which, I think,… https://t.co/SPZmyDh5Ry',
       'RT @Harlina__: @cafedotcom @tedcruz the whole POTUS is a joke . Declining education , climate change and congratulating people for being si…',
       '@LibertyUSA1776 @c5hardtop1999 I guarantee you that if you asked any of the refugees none will say $q$climate change$q$ is the reason I left.',
       ...,
       'Climate Depot:Cl-FI: New Study apes Hollywood: ‘Global warming can trigger a cooling trend’ – Climate scientist http://t.co/4ikBwkk0nD',
       "RT @TeslaMotors: Rising temperatures put millions at risk as climate change hotspot @third_pole (world's 3rd-largest store of ice) is meltiÃ¢â‚¬Â¦",
       "RT @RandomSavage3: @FrMatthewLC Crazy how two days ago people were going bezerk about 'global warming' then a blizzard hits"],
      dtype=object)

In [15]:

from tensorflow.keras import layers
import tensorflow as tf
tf.random.set_seed(42)
inputs=layers.Input(shape=(1,),dtype="string")
x=text_vectorizer(inputs)

x=embedding(x)
x=layers.Bidirectional(layers.GRU(64,return_sequences=True))(x) #layers.Bidirectional works with any LSTM
x=layers.GRU(64)(x)
outputs=layers.Dense(4,activation="softmax")(x)
model_4=tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [16]:
model_4.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])


# Fit the model
model_4_history=model_4.fit(train_sentences,
                            train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels))


Epoch 1/5
962/962 [==============================] - 41s 30ms/step - loss: 0.8237 - accuracy: 0.6700 - val_loss: 0.6809 - val_accuracy: 0.7221
Epoch 2/5
962/962 [==============================] - 14s 14ms/step - loss: 0.5532 - accuracy: 0.7851 - val_loss: 0.6882 - val_accuracy: 0.7234
Epoch 3/5
962/962 [==============================] - 12s 13ms/step - loss: 0.4155 - accuracy: 0.8409 - val_loss: 0.7703 - val_accuracy: 0.7311
Epoch 4/5
962/962 [==============================] - 13s 14ms/step - loss: 0.3106 - accuracy: 0.8853 - val_loss: 0.8440 - val_accuracy: 0.7203
Epoch 5/5
962/962 [==============================] - 12s 12ms/step - loss: 0.2337 - accuracy: 0.9156 - val_loss: 0.9534 - val_accuracy: 0.7137


In [20]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.5 MB/s eta 0:00:00


In [46]:
import tensorflow_hub as hub
embed=hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
embed_sample=embed([
                    "When you call the universal sentence encoder on a sentence, it turns it into numbers."])


embed_sample

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[ 3.59669030e-02, -8.57946724e-02, -1.15274135e-02,
         5.25983237e-03, -1.85217280e-02, -5.04201241e-02,
        -3.61694060e-02,  5.34677925e-03,  4.80591506e-02,
         4.69074398e-02, -3.72333117e-02, -1.14954300e-02,
         4.35241461e-02,  7.05099106e-02,  7.09376186e-02,
        -8.18042904e-02,  8.71716533e-03, -4.65412140e-02,
        -2.24577412e-02,  4.68687154e-02,  2.02256138e-03,
         3.09907366e-02,  2.04356425e-02,  6.39216825e-02,
        -7.64108524e-02,  8.42117816e-02, -4.57603931e-02,
        -1.06165220e-03, -2.05942076e-02,  1.24110505e-02,
         5.72753772e-02,  3.81562151e-02, -2.74211336e-02,
        -3.54347005e-03, -9.83258486e-02, -1.24485493e-02,
         3.86562832e-02,  5.03195487e-02, -2.36250404e-02,
         3.21848365e-03,  3.22521329e-02,  7.38092232e-03,
         4.47310135e-02, -4.12239181e-03,  1.15160448e-02,
         2.83772685e-02,  6.01376989e-04, -5.90335056e-02,
      

In [64]:
sentence_encoder_layer=hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                                                    input_shape=[],
                                                                    dtype=tf.string,
                                                                    trainable=True,
                                                                    name="USE"
                                                                    )


In [65]:
from tensorflow.keras import layers
import tensorflow as tf
tf.random.set_seed(42)
model_6=tf.keras.Sequential([
    sentence_encoder_layer,
    # tf.keras.layers.Reshape((1, -1)),
    # tf.keras.layers.GRU(64),
    tf.keras.layers.Dense(4,activation='softmax')
],name='model_6_USE')
model_6.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                            optimizer=tf.keras.optimizers.Adam(0.00001),
                            metrics=['accuracy']
                                )
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_20 (Dense)            (None, 4)                 2052      
                                                                 
Total params: 256,799,876
Trainable params: 256,799,876
Non-trainable params: 0
_________________________________________________________________


In [68]:
import numpy as np
train_10_percent_split=int(0.5*len(train_sentences))
train_sentences_10_percent=train_sentences[:train_10_percent_split]
train_labels_10_percent=train_labels[:train_10_percent_split]

pd.Series(np.array(train_labels_10_percent)).value_counts()

1    8016
2    3291
0    2635
3    1438
dtype: int64

In [ ]:
model_6_history=model_6.fit(x=train_sentences_10_percent,
                              y=train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                            )

Epoch 1/5
481/481 [==============================] - 53s 111ms/step - loss: 0.3478 - accuracy: 0.8799 - val_loss: 0.6882 - val_accuracy: 0.7315
Epoch 2/5
481/481 [==============================] - 52s 108ms/step - loss: 0.0322 - accuracy: 0.9902 - val_loss: 0.9548 - val_accuracy: 0.7171
Epoch 3/5
481/481 [==============================] - 49s 102ms/step - loss: 0.0082 - accuracy: 0.9983 - val_loss: 1.0031 - val_accuracy: 0.7254
Epoch 4/5
481/481 [==============================] - 59s 122ms/step - loss: 0.0036 - accuracy: 0.9993 - val_loss: 1.1119 - val_accuracy: 0.7348
Epoch 5/5
481/481 [==============================] - 50s 104ms/step - loss: 0.0023 - accuracy: 0.9997 - val_loss: 1.1793 - val_accuracy: 0.7154
